In [125]:
from bs4 import BeautifulSoup
import os 
import re
import nltk
from copy import deepcopy

ECTNestedDict = {}
file_path = os.getcwd() + "/ECT/"

file_list = os.listdir(file_path)

# Dictionary Construction
i=0
path = file_path + file_list[i]
ID = re.findall(r'\d+', file_list[i])[0]
print("DocID: " + ID)


tr_file = open(path)
# Parsing the html
soup = BeautifulSoup(tr_file, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# Storing Date
# Storing the 1st line
# a = (soup.find('p', class_='p p1')).get_text()  # For finding a specific class in the html
a = (soup.find('p', {'class' : 'p p1'})).get_text()  # For finding a specific class in the html
#  Finding the date through regex
a = re.findall(r'[A-Z]\w+\s+\d+\,\s+\d+',a)
# Removing extra spaces from the date
date = " ".join(a[0].split())

a = soup.findAll('p')

# Constructing Participant List
part_list= []
flag=0  # flag for storing no. of strong texts <strong> texts
ctr=-1

while(flag<3):
    ctr+=1
    tag = a[ctr]
    if(tag.find('strong')):
        flag+=1
    elif(flag>0):
        part_list.append(tag.get_text())

presentation_dict ={}
temp =[]
speaker = ""
flag = 0
qna_head_phrases = ["question-and-answer session", "question-and-answers session", "questions-and-answers session", "questions-and-answer session"]

while(ctr<len(a) and ((a[ctr].get_text()).lower() not in qna_head_phrases)):
    tag = a[ctr]
    if(tag.find('strong')):
        if((tag.get_text() not in presentation_dict.keys())):
            if(flag==0):
                speaker = tag.get_text()
                flag=1
            elif(flag!=0):
                presentation_dict[speaker] = temp
                temp=[]
                speaker = tag.get_text()
        else:
            presentation_dict[speaker] = temp
            speaker = tag.get_text()
            temp = presentation_dict[speaker]
    else:
        temp.append(tag.get_text())
    ctr+=1

ctr+=1 # To move forward from the Q-&-A Heading tag

# Construction of Questionnaire Dictionary
question_dict = {}
temp = {}
speaker = ""
flag = 0
remarks = []
flow_ctr= 1
while(ctr<len(a)):
    tag =a[ctr]
    if((tag.find('strong') or (tag.get_text() in part_list))):
        if(flag==0):
            speaker = tag.get_text()
            flag=1
        else:
            temp["Speaker"] = deepcopy(speaker)
            temp["Remarks"] = remarks
            question_dict[flow_ctr] = deepcopy(temp)
            speaker = tag.get_text()
            remarks = []
            temp = {}
            flow_ctr+=1
    else:
        remarks.append(tag.get_text())
    ctr+=1

# Added to add the last speaker to the dict.
temp["Speaker"] = deepcopy(speaker)
temp["Remarks"] = remarks
question_dict[flow_ctr] = deepcopy(temp)


DocID: 531


In [ ]:
print(date)
for temp in part_list:
    print

In [ ]:

for i in range(1, len(question_dict.keys())+1):
    temp = question_dict[i]
    print(str(i) + ": " + temp["Speaker"])
    for b in temp["Remarks"]:
        print(b)

In [ ]:
a = (soup.findAll('strong'))
Question-And-Answer Session

# For finding with ID
# soup.find(id='abc1')